In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import spacy

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cluster import KMeans
from sklearn.utils import class_weight
from sklearn.utils import resample
from sklearn.pipeline import Pipeline


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Dropout, Conv1D, MaxPooling1D, Flatten, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Bidirectional, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [2]:
# Cargar datos
df_procesados = pd.read_csv('../data/datos_nlp_procesados_2.csv')

# Filtrar clases muy pequeñas (opcional, para evitar errores si hay solo 1 ejemplo)
conteo = df_procesados['especialidad_corr'].value_counts()
clases_validas = conteo[conteo > 5].index
df_procesados = df_procesados[df_procesados['especialidad_corr'].isin(clases_validas)]

# Convertir etiquetas de texto a números (Cardiología -> 0, Respiratorio -> 1...)
label_encoder = LabelEncoder()
df_procesados['label_num'] = label_encoder.fit_transform(df_procesados['especialidad_corr'])
# Separar datos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(
    df_procesados['sintomas_procesados'], 
    df_procesados['label_num'], 
    test_size=0.20, 
    random_state=42,
    stratify=df_procesados['label_num'] # Mantiene la proporción de clases
)

print(f"Datos listos. Entrenamiento: {len(X_train)} filas. Prueba: {len(X_test)} filas.")
print("Clases detectadas:", len(label_encoder.classes_))

# Mostrar las clases
for i, clase in enumerate(label_encoder.classes_):
    print(f"{i}: {clase}")

Datos listos. Entrenamiento: 5537 filas. Prueba: 1385 filas.
Clases detectadas: 12
0: CARDIOLOGÍA/CIRCULATORIO
1: DERMATOLOGÍA
2: ENDOCRINOLOGÍA/NUTRICIÓN
3: GASTROENTEROLOGÍA/DIGESTIVO
4: GINECOLOGÍA/OBSTETRICIA
5: NEUROLOGÍA
6: OFTALMOLOGÍA/ORL
7: ONCOLOGÍA (TUMORES)
8: PSIQUIATRÍA/MENTAL
9: SÍNTOMAS GENERALES/NO CLASIFICADOS
10: TRAUMATOLOGÍA/MUSCULAR
11: UROLOGÍA/RENAL


## Modelo SVM

In [ ]:
# 1. Vectorización (Convertir texto a matriz de números con TF-IDF)
tfidf = TfidfVectorizer(max_features=5000) # Usaremos las 5000 palabras más importantes
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

# 2. Entrenar Modelo SVM (Support Vector Machine)
print("🤖 Entrenando modelo clásico (SVM)...")
svm_model = SVC(kernel='linear', random_state=42, class_weight='balanced')
svm_model.fit(X_train_tfidf, y_train)

# 3. Evaluar
y_pred_svm = svm_model.predict(X_test_tfidf)
acc_svm = accuracy_score(y_test, y_pred_svm)

print(f"✅ Precisión del Modelo Clásico (SVM): {acc_svm*100:.2f}%")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_svm, target_names=label_encoder.classes_, zero_division=0))

In [ ]:
# ==========================================
# 🚀 ESTRATEGIA AVANZADA PARA SVM (>80%)
# ==========================================

print("🤖 Iniciando optimización avanzada del SVM (Esto puede tardar unos minutos)...")

# 1. Definimos un 'Pipeline' (Tubería de procesos)
# Esto asegura que el preprocesamiento y el modelo viajen juntos
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        strip_accents='unicode',
        lowercase=True
    )),
    ('svm', SVC(class_weight='balanced', probability=True))
])

# 2. Definimos la 'Rejilla de Hiperparámetros' (El menú de opciones a probar)
# La IA probará todas estas combinaciones para encontrar la ganadora
param_grid = {
    # Búsqueda de N-Grams:
    # (1,1) = Solo palabras sueltas
    # (1,2) = Palabras sueltas Y parejas ("dolor de", "de pecho") -> CRÍTICO EN MEDICINA
    'tfidf__ngram_range': [(1, 2)], 
    
    # Limpieza de vocabulario:
    # min_df=2: Ignora palabras que aparecen en menos de 2 documentos (errores tipográficos)
    'tfidf__min_df': [2, 3],
    
    # max_features: Probamos con más palabras o sin límite
    'tfidf__max_features': [5000, 7000, None],
    
    # Parámetros del SVM:
    # C: Qué tan estricto es el modelo (Valores altos = menos errores en train, riesgo de overfitting)
    # kernel: La forma matemática de separar los datos
    'svm__C': [1, 10, 100],
    'svm__kernel': ['linear', 'rbf'] 
}

# 3. Ejecutar GridSearch (Fuerza Bruta Inteligente)
# cv=5 significa Validación Cruzada de 5 pliegues (entrena 5 veces con distintos trozos de datos)
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1, scoring='accuracy')

# Entrenar
grid_search.fit(X_train, y_train)

# ==========================================
# 🏆 RESULTADOS
# ==========================================
print(f"\n✅ ¡Optimización completada!")
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")
print(f"Mejor precisión en validación cruzada: {grid_search.best_score_*100:.2f}%")

# Guardamos el mejor modelo
best_svm = grid_search.best_estimator_

# 4. Evaluación Final en el set de Prueba (Test)
y_pred_svm = best_svm.predict(X_test)
acc_svm = accuracy_score(y_test, y_pred_svm)

print(f"\n🏆 PRECISIÓN FINAL EN TEST: {acc_svm*100:.2f}%")
print("\nReporte de Clasificación Detallado:")
print(classification_report(y_test, y_pred_svm, target_names=label_encoder.classes_, zero_division=0))

In [ ]:
# Guardar el modelo mejorado
with open('../models/modelo_svm_optimizado.pkl', 'wb') as f:
    pickle.dump(best_svm, f)
print("💾 Modelo optimizado guardado en '../models/modelo_svm_optimizado.pkl'")

In [3]:
# ==========================================
# 🏆 ENTRENAMIENTO FINAL (MEJOR MODELO)
# ==========================================

# Usamos los parámetros ganadores del GridSearch anterior
pipeline_final = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1, 2),    # Bigramas (Clave)
        min_df=2,              # Ignorar errores únicos
        max_features=None,     # Usar todo el vocabulario
        strip_accents='unicode',
        lowercase=True
    )),
    ('svm', SVC(
        C=100,                 # Margen estricto
        kernel='linear',       # Separación lineal
        class_weight='balanced', 
        probability=True       # Para ver % de confianza en la demo
    ))
])

print("🚀 Entrenando Modelo Final con clases corregidas...")
pipeline_final.fit(X_train, y_train)

# Evaluación
y_pred = pipeline_final.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"\n🌟 PRECISIÓN FINAL: {acc*100:.2f}%")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

🚀 Entrenando Modelo Final con clases corregidas...

🌟 PRECISIÓN FINAL: 82.45%
                                    precision    recall  f1-score   support

          CARDIOLOGÍA/CIRCULATORIO       0.94      0.91      0.92       229
                      DERMATOLOGÍA       0.91      0.67      0.77        15
          ENDOCRINOLOGÍA/NUTRICIÓN       0.74      0.77      0.76        22
       GASTROENTEROLOGÍA/DIGESTIVO       0.90      0.89      0.89       142
           GINECOLOGÍA/OBSTETRICIA       0.90      0.89      0.89        62
                        NEUROLOGÍA       0.71      0.76      0.73       185
                  OFTALMOLOGÍA/ORL       0.89      0.95      0.92        96
               ONCOLOGÍA (TUMORES)       0.53      0.64      0.58        42
                PSIQUIATRÍA/MENTAL       0.84      0.84      0.84        25
SÍNTOMAS GENERALES/NO CLASIFICADOS       0.68      0.75      0.71       154
            TRAUMATOLOGÍA/MUSCULAR       0.84      0.79      0.82       281
         

In [4]:
# Guardar
with open('../models/modelo_svm_final_80.pkl', 'wb') as f:
    pickle.dump(pipeline_final, f)
    
# Guardar también el nuevo encoder (¡Muy importante porque cambiamos las clases!)
with open('../models/label_encoder_final_80.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [ ]:
# ==========================================
# 🏆 DEMO FINAL: USANDO EL MODELO (SVM)
# ==========================================

# Cargar el modelo guardado
with open('../models/modelo_svm_final_80.pkl', 'rb') as f:
    modelo_cargado = pickle.load(f)
# Cargar el label encoder guardado
with open('../models/label_encoder_final_80.pkl', 'rb') as f:
    label_encoder_cargado = pickle.load(f)

# Ejemplo de uso
def predecir_especialidad(sintomas):
    prediccion_num = modelo_cargado.predict([sintomas])[0]
    prediccion_texto = label_encoder_cargado.inverse_transform([prediccion_num])[0]
    return prediccion_texto
ejemplo_sintomas = ["Paciente con dolor torácico intenso, dificultad para respirar y sudoración profusa.",
                    "Fiebre alta, tos persistente y dolor de garganta desde hace tres días.",
                    "Dolor abdominal severo, náuseas y vómitos después de comer.",
                    "Mareos, visión borrosa y debilidad en las extremidades.",
                    "Erupción cutánea con picazón intensa y ampollas en varias partes del cuerpo.",
                    "Dolor de cabeza intenso, rigidez en el cuello y sensibilidad a la luz.",
                    "Dolor en las articulaciones, hinchazón y dificultad para moverse.",
                    "Dificultad para respirar, sibilancias y opresión en el pecho.",
                    "Dolor lumbar severo, entumecimiento en las piernas y pérdida de control de la vejiga.",
                    "Fatiga extrema, pérdida de peso inexplicada y sudores nocturnos.",
                    ]
for sintomas in ejemplo_sintomas:
    especialidad_predicha = predecir_especialidad(sintomas)
    print(f"Sintomas: {sintomas}\n➡️ Especialidad Predicha: {especialidad_predicha}\n")


In [8]:
# ==========================================
# 🏆 DEMO FINAL: USANDO EL MODELO (SVM)
# ==========================================
# Nota: Usamos el SVM porque demostró ser más robusto con la cantidad actual de datos.
try:
    nlp = spacy.load("es_core_news_sm")
except:
    import os
    os.system("python -m spacy download es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

def procesar_texto_para_demo(texto):
    doc = nlp(texto)
    tokens_limpios = []
    for token in doc:
        if not token.is_punct and not token.is_stop and token.is_alpha:
            tokens_limpios.append(token.lemma_.lower())
    return " ".join(tokens_limpios)

def predecir_con_svm(texto_usuario):
    # 1. Limpieza (Igual que siempre)
    texto_limpio = procesar_texto_para_demo(texto_usuario)
    
    if not texto_limpio:
        print("⚠️ Escribe algo con sentido médico.")
        return

    # 2. Vectorización (Usamos TF-IDF en lugar del Tokenizer)
    # Importante: Usamos .transform(), NO .fit_transform()
    vector_numerico = tfidf.transform([texto_limpio]).toarray()
    
    # 3. Predicción con SVM
    prediccion_index = svm_model.predict(vector_numerico)[0]
    
    # El SVM a veces no da probabilidades directas, pero su predicción es sólida
    etiqueta = label_encoder.inverse_transform([prediccion_index])[0]
    
    # 4. Mostrar resultado
    print(f"👤 Usuario: '{texto_usuario}'")
    print(f"⚙️ Procesado: '{texto_limpio}'")
    print(f"🏥 Especialidad: {etiqueta.upper()}")
    print("-" * 40)

# --- PRUEBA FINAL ---
print("🚀 Iniciando Triaje con Modelo SVM (Campeón)...\n")

predecir_con_svm("El paciente presenta dolor fuerte en el pecho y dificultad para respirar")
predecir_con_svm("Mancha roja en la piel que pica y arde")
predecir_con_svm("Pérdida de visión en el ojo derecho borroso")
predecir_con_svm("Fractura de hueso por caída fuerte")
predecir_con_svm("Ardor al orinar y dolor en los riñones")
predecir_con_svm("El paciente presenta dolor fuerte en el pecho y dificultad para respirar")
predecir_con_svm("Tengo una mancha roja en la piel que me pica mucho")
predecir_con_svm("Dolor intenso en el ojo derecho y visión borrosa")
predecir_con_svm("Fractura en la pierna tras caída")
predecir_con_svm("Ardor al orinar y dolor en los riñones")
predecir_con_svm("Dolor en la cabeza y náuseas")
predecir_con_svm("Vomito constante y fiebre alta")
predecir_con_svm("Necesito ayuda inmediata, no puedo respirar bien")
predecir_con_svm("Dolor abdominal severo y vómitos persistentes")
predecir_con_svm("Fatiga extrema y mareos al levantarme")
predecir_con_svm("Siento un dolor agudo en el oído y pérdida de audición")
predecir_con_svm("Tengo fiebre alta y dolor de garganta")
predecir_con_svm("Me duele mucho la espalda baja y no puedo moverme bien")
predecir_con_svm("Tengo una herida profunda en la mano que no deja de sangrar")
predecir_con_svm("Estoy teniendo dificultad para hablar y debilidad en un lado del cuerpo")
predecir_con_svm("Me salieron unas machas moradas en la piel y me siento muy cansado")
predecir_con_svm("Tengo llagas en la boca que me duelen mucho al comer")

🚀 Iniciando Triaje con Modelo SVM (Campeón)...



NameError: name 'tfidf' is not defined

## Modelo Deep Learning

In [ ]:
# Cargar embeddings pre-entrenados de FastText
EMBEDDING_FILE = '../models/cc.es.300.vec'

# Cargar el vocabulario de FastText en memoria (solo lo necesario)
print("Cargando embeddings pre-entrenados...")
embeddings_index = {}
with open(EMBEDDING_FILE, encoding='utf-8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'Se encontraron {len(embeddings_index)} vectores de palabras en FastText.')

In [ ]:
# ==============================================
# 🚀 ENTRENAMIENTO DEL MODELO DE DEEP LEARNING
# ==============================================

# --- 1. CONFIGURACIÓN ---
VOCAB_SIZE = 5000      # Vocabulario
EMBEDDING_DIM = 300    # Dimensión FastText (¡Debe ser 300!)
MAX_LENGTH = 100       # Longitud de frase

print("⚙️ Configuración cargada.")

# --- 2. PREPARACIÓN DE EMBEDDINGS (MATRIZ DE PESOS) ---
# Verificamos si ya tienes los embeddings cargados para no repetir el proceso pesado
if 'embeddings_index' not in globals():
    raise ValueError("⚠️ Por favor, carga el archivo .vec de FastText en la variable 'embeddings_index' antes de ejecutar este bloque.")

# Matriz de ceros inicial
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))
hits = 0
misses = 0

# Tokenizer: Aprende el vocabulario de tus datos
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

# Rellenar matriz con FastText
for word, i in word_index.items():
    if i < VOCAB_SIZE:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1

print(f"✅ Matriz de Embeddings lista. Hits: {hits} | Misses: {misses}")

# --- 3. PREPARACIÓN DE DATOS (SECUENCIAS) ---
train_seq = tokenizer.texts_to_sequences(X_train)
test_seq = tokenizer.texts_to_sequences(X_test)

train_padded = pad_sequences(train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
test_padded = pad_sequences(test_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

# Pesos para clases desbalanceadas (Crucial para que no ignore clases pequeñas)
pesos = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
pesos_dict = dict(enumerate(pesos))
print("⚖️ Pesos de clase calculados.")

# --- 4. CONSTRUCCIÓN DEL MODELO ---
model = Sequential([
    Input(shape=(MAX_LENGTH,)),
    
    # Capa de Embedding (Inicialmente CONGELADA)
    Embedding(input_dim=VOCAB_SIZE, 
              output_dim=EMBEDDING_DIM, 
              weights=[embedding_matrix], 
              trainable=False,  # <--- FASE 1: NO TOCAR
              name="embedding_layer"),
    
    # Dropout espacial: apaga palabras enteras, no solo neuronas, fuerza a entender contexto
    SpatialDropout1D(0.3),
    
    # LSTM Bidireccional potente
    Bidirectional(LSTM(128, return_sequences=False)),
    
    # Clasificador
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y_train)), activation='softmax')
])

model.summary()

# --- 5. CALLBACKS INTELIGENTES ---
# Detiene si no mejora en 5 épocas
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# Reduce la velocidad de aprendizaje si se estanca (Ayuda a bajar el loss suavemente)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)

# ==========================================
# 🚀 FASE 1: ENTRENAMIENTO GENERAL (CEREBRO)
# ==========================================
print("\n🔵 FASE 1: Entrenando LSTM con Embeddings Congelados...")
optimizer = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history1 = model.fit(
    train_padded, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(test_padded, y_test),
    class_weight=pesos_dict,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ==========================================
# 🚀 FASE 2: FINE-TUNING (AJUSTE FINO)
# ==========================================
print("\n🟢 FASE 2: Descongelando Embeddings para Ajuste Fino (La magia)...")

# 1. Descongelamos la capa de Embedding
model.get_layer("embedding_layer").trainable = True

# 2. Recompilamos con un Learning Rate MUY BAJO (1e-5)
# Esto permite modificar los vectores de palabras suavemente sin romper lo aprendido
optimizer_fine = Adam(learning_rate=1e-5)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer_fine, metrics=['accuracy'])

# 3. Entrenamos de nuevo (Fine-Tuning)
history2 = model.fit(
    train_padded, y_train,
    epochs=15,
    batch_size=32,
    validation_data=(test_padded, y_test),
    class_weight=pesos_dict,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

print("\n🏆 Entrenamiento Completado.")

In [ ]:
# Guardar el modelo
model.save('../models/mi_red_neuronal_v5.h5')

with open('../models/tokenizer_v5.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../models/label_encoder_v5.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("✅ Modelo de Red Neuronal entrenado y guardado.")

In [ ]:
# Gráfico de Aprendizaje
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history1.history['accuracy'], label='Entrenamiento')
plt.plot(history1.history['val_accuracy'], label='Validación (Prueba)')
plt.title('Precisión de la Red Neuronal durante el entrenamiento')
plt.xlabel('Épocas')
plt.legend()

# Comparación Final
loss, acc_dl = model.evaluate(test_padded, y_test, verbose=0)
print(f"\n🏆 RESUMEN FINAL:")
print(f"Precisión Machine Learning (SVM): {acc_svm*100:.2f}%")
print(f"Precisión Deep Learning (Red Neuronal): {acc_dl*100:.2f}%")

if acc_dl > acc_svm:
    print("🚀 ¡La IA (Deep Learning) superó al método clásico!")
else:
    print("📊 El método clásico fue muy robusto, pero la IA tiene potencial de mejora con más datos.")

In [ ]:
print("🔍 Ejecutando Aprendizaje No Supervisado (K-Means)...")

# Queremos agrupar en tantos grupos como especialidades creemos que hay (o un numero arbitrario ej: 5)
num_clusters = 12
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)

# Ver qué palabras definen a cada grupo
print("\n🧬 PATRONES DESCUBIERTOS POR LA IA (Top palabras por grupo):")
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = tfidf.get_feature_names_out()

diccionario_clusters = {}

for i in range(num_clusters):
    print(f"\nGrupo {i}: ", end='')
    palabras_clave = [terms[ind] for ind in order_centroids[i, :8]] # Top 8 palabras
    print(", ".join(palabras_clave))
    diccionario_clusters[i] = palabras_clave

print("\n✅ Clustering completado. Estos grupos se formaron matemáticamente sin ayuda humana.")

In [ ]:
# ==========================================
# 🚑 DEMO INTERACTIVA: TRIAJE 593
# ==========================================

try:
    nlp = spacy.load("es_core_news_sm")
except:
    import os
    os.system("python -m spacy download es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

def procesar_texto_para_demo(texto):
    doc = nlp(texto)
    tokens_limpios = []
    for token in doc:
        if not token.is_punct and not token.is_stop and token.is_alpha:
            tokens_limpios.append(token.lemma_.lower())
    return " ".join(tokens_limpios)

def predecir_urgencia(texto_usuario):
    # 1. Limpieza
    texto_limpio = procesar_texto_para_demo(texto_usuario)
    if not texto_limpio: # Si el usuario pone solo simbolos o stopwords
        print("⚠️ Por favor escribe una frase médica válida.")
        return
    seq = tokenizer.texts_to_sequences([texto_limpio])
    padded = pad_sequences(seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
    
    # 2. Predicción con la Red Neuronal
    prediccion = model.predict(padded, verbose=0)
    
    # 3. Decodificar el resultado
    clase_predicha_index = np.argmax(prediccion)
    probabilidad = np.max(prediccion)
    etiqueta = label_encoder.inverse_transform([clase_predicha_index])[0]
    
    # 4. Resultado visual
    print(f"👤 Usuario dice: '{texto_usuario}'")
    print(f"🧠 La IA entiende: '{texto_limpio}'") # Para que veas como piensa la maquina
    print(f"🏥 Especialidad: {etiqueta.upper()}")
    print(f"📊 Confianza: {probabilidad*100:.2f}%")
    print("-" * 40)

# --- PRUEBA DE PROYECTO ---
print("🚀 Iniciando pruebas de Triaje Inteligente...\n")

# Casos de prueba (Pruébalos y luego cambia el texto por lo que quieras)
predecir_urgencia("El paciente presenta dolor fuerte en el pecho y dificultad para respirar")
predecir_urgencia("Tengo una mancha roja en la piel que me pica mucho")
predecir_urgencia("Dolor intenso en el ojo derecho y visión borrosa")
predecir_urgencia("Fractura en la pierna tras caída")
predecir_urgencia("Ardor al orinar y dolor en los riñones")
predecir_urgencia("Dolor en la cabeza y náuseas")
predecir_urgencia("Vomito constante y fiebre alta")
predecir_urgencia("Necesito ayuda inmediata, no puedo respirar bien")
predecir_urgencia("Dolor abdominal severo y vómitos persistentes")
predecir_urgencia("Fatiga extrema y mareos al levantarme")
predecir_urgencia("Siento un dolor agudo en el oído y pérdida de audición")
predecir_urgencia("Tengo fiebre alta y dolor de garganta")
predecir_urgencia("Me duele mucho la espalda baja y no puedo moverme bien")
predecir_urgencia("Tengo una herida profunda en la mano que no deja de sangrar")
predecir_urgencia("Estoy teniendo dificultad para hablar y debilidad en un lado del cuerpo")
predecir_urgencia("Me salieron unas machas moradas en la piel y me siento muy cansado")
predecir_urgencia("Tengo llagas en la boca que me duelen mucho al comer")